In [ ]:
from __future__ import division, print_function, unicode_literals    #All Libraries
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler
import torch.nn as nn
from torch.autograd import Variable
from torchsummary import summary
import os
import pandas as pd
import scipy 
from timeit import default_timer as timer
from torch.optim import lr_scheduler
%matplotlib inline
plt.ion()
import xml.etree.ElementTree as xml
from pprint import PrettyPrinter
import time
import cv2
from PIL import *
import matplotlib.patches as patches
from os import path
import cv2
from collections import defaultdict
import imutils
import glob 
import random

In [ ]:
#Building the Dataset for Train and Val

In [ ]:
def get_rand_bbox(img):
      rsize = 50 + random.randint(0,100)
      rxmin = random.randint(0, img.shape[1]-rsize-3)
      rxmax = rxmin + rsize
      rymin = random.randint(0, img.shape[0]-rsize-3)
      rymax = rymin + rsize
      bbox = [rxmin, rymin, rxmax, rymax]
      return bbox
def IoU(box1, box2):
    # determine the (x, y)-coordinates of the intersection rectangle
    [x1_min, x1_max, y1_min, y1_max] = box1
    [x2_min, x2_max, y2_min, y2_max] = box2
    # point A is bottom left corner and point B is top right point of intersection
    xA = max(x1_min, x2_min)
    xB = min(x1_max, x2_max)
    yA = max(y1_min, y2_min)
    yB = min(y1_max, y2_max)
    if xA > xB or yA > yB:
        return 0

    intersection = (xB - xA) * (yB - yA)

    box1Area = (x1_max - x1_min) * (y1_max - y1_min)
    box2Area = (x2_max - x2_min) * (y2_max - y2_min)

    iou = intersection / float(box1Area + box2Area - intersection)
    return iou

In [ ]:
#Making Folders for saving images
file_path = "/home/suda/Desktop/val2"
if not os.path.exists(file_path):
    os.mkdir(file_path)
file_path = "/home/suda/Desktop/train2"
if not os.path.exists(file_path):
    os.mkdir(file_path)
file_path = "/home/suda/Desktop/test2"
if not os.path.exists(file_path):
    os.mkdir(file_path)
    
    
    
file_path = "/home/suda/Desktop/val2/bottle"
if not os.path.exists(file_path):
    os.mkdir(file_path)
file_path = "/home/suda/Desktop/val2/_background_"
if not os.path.exists(file_path):
    os.mkdir(file_path)
file_path = "/home/suda/Desktop/val2/aeroplane"
if not os.path.exists(file_path):
    os.mkdir(file_path)
file_path = "/home/suda/Desktop/val2/chair"
if not os.path.exists(file_path):
    os.mkdir(file_path)
    
file_path = "/home/suda/Desktop/train2/bottle"
if not os.path.exists(file_path):
    os.mkdir(file_path)
file_path = "/home/suda/Desktop/train2/_background_"
if not os.path.exists(file_path):
    os.mkdir(file_path)
file_path = "/home/suda/Desktop/train2/aeroplane"
if not os.path.exists(file_path):
    os.mkdir(file_path)
file_path = "/home/suda/Desktop/train2/chair"
if not os.path.exists(file_path):
    os.mkdir(file_path)
    
    
file_path = "/home/suda/Desktop/test2/bottle"
if not os.path.exists(file_path):
    os.mkdir(file_path)
file_path = "/home/suda/Desktop/test2/_background_"
if not os.path.exists(file_path):
    os.mkdir(file_path)
file_path = "/home/suda/Desktop/test2/aeroplane"
if not os.path.exists(file_path):
    os.mkdir(file_path)
file_path = "/home/suda/Desktop/test2/chair"
if not os.path.exists(file_path):
    os.mkdir(file_path)
    
    

In [ ]:
counter2 = 0
counter = 0


flag = 1
annot_dir ='/home/suda/Desktop/train/VOC2007/Annotations/'
image_dir ='/home/suda/Desktop/train/VOC2007/JPEGImages/'


t = glob.iglob(annot_dir+'/*')
h=  os.listdir(annot_dir)

for annot_file in t:
    counter2 += 1
    if counter2%500 == 1:
        print(counter2)


    file = open(annot_file,'r')
    parsed = xml.parse(file)
    root = parsed.getroot()

    for child in root.iter('filename'):
        img_file = child.text
#         print(img_file)

    img = cv2.imread(image_dir + img_file)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    objects = []
    labels = []

    for object in root.iter('object'):
        if object[0].text in classes:
            xmin = int(float(object[4][0].text))
            ymin = int(float(object[4][1].text))
            xmax = int(float(object[4][2].text))
            ymax = int(float(object[4][3].text))
            bbox = [xmin, ymin, xmax, ymax]

            objects.append(bbox)
            labels.append(object[0].text)

    # find random background from the same image
    #  for each 3 objects find 1 object; 0 means NO, 1 means YES
    for idx in range(len(objects)):
        do_take = np.random.choice([0,1], p=[0.6,0.4])
        threshold = 0.2

        if do_take == 1:
            taken = False
            img_contour = [img.shape[1],img.shape[0]] * 2

            try_count = 0
            while not taken:
                try_count += 1
                if try_count == 5:
                    break
                valid = True
                rand_bbox = get_rand_bbox(img)
                for bbox in objects:
                    if IoU(bbox, rand_bbox) > threshold:
                        valid = False
                        break

                if valid == True:
                    objects.append(rand_bbox)
                    labels.append('_background_')
                    taken = True

    for idx_, object_ in enumerate(objects):
        try:
            img_object = img[object_[1]:object_[3],object_[0]:object_[2]]
            img_object = cv2.resize(img_object, (224,224))
            counter = counter + 1
            prefix_train = '/home/suda/Desktop/train2/'
            prefix_val = '/home/suda/Desktop/val2/'
            if labels[idx_]=="bottle" :
                if flag % 4 == 1:
                    cv2.imwrite(prefix_val+labels[idx_]+'/b'+str(counter)+'.jpg',img_object)
                    continue
                else:
                    cv2.imwrite(prefix_train+labels[idx_]+'/b'+str(counter)+'.jpg',img_object)
            flag+=1

            if labels[idx_]=="aeroplane" :
                if flag % 3 == 1:
                    cv2.imwrite(prefix_val+labels[idx_]+str(counter)+'.jpg',img_object)
                    cv2.imwrite(prefix_val+labels[idx_]+'/a'+str(counter)+'.jpg',img_object)
                    cv2.imwrite(prefix_val+labels[idx_]+'/b'+str(counter)+'.jpg',img_object)
                    continue
                else:
                    cv2.imwrite(prefix_train+labels[idx_]+'/b'+str(counter)+'.jpg',img_object)
            flag+=1
        except:
            pass

In [ ]:
#Building the Dataset for Test

In [ ]:
counter2 = 0
counter = 0

flag = 1
annot_dir ='/home/suda/Desktop/test/VOC2007/Annotations/'
image_dir ='/home/suda/Desktop/test/VOC2007/JPEGImages/'
print("A")
t = glob.iglob(annot_dir+'/*')
h= os.listdir(annot_dir)

print(t)
type(t)
for annot_file in t:

    counter2 += 1
    if counter2%500 == 1:
        print(counter2)

    #print("B")

    file = open(annot_file,'r')
    parsed = xml.parse(file)
    root = parsed.getroot()

    for child in root.iter('filename'):
        img_file = child.text

    img = cv2.imread(image_dir + img_file)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

    objects = []
    labels = []

    for object in root.iter('object'):
        if object[0].text in classes:
            xmin = int(float(object[4][0].text))
            ymin = int(float(object[4][1].text))
            xmax = int(float(object[4][2].text))
            ymax = int(float(object[4][3].text))
            bbox = [xmin, ymin, xmax, ymax]

            objects.append(bbox)
            labels.append(object[0].text)

    # find random background from the same image
    # for each 3 objects find 1 object; 0 means NO, 1 means YES
    for idx in range(len(objects)):
        do_take = np.random.choice([0,1], p=[0.6,0.4])
        threshold = 0.1

        if do_take == 1:
            taken = False
            img_contour = [img.shape[1],img.shape[0]] * 2

            try_count = 0
            while not taken:
                try_count += 1
                if try_count == 5:
                    break
                valid = True
                rand_bbox = get_rand_bbox(img)
                for bbox in objects:
                    if IoU(bbox, rand_bbox) > threshold:
                        valid = False
                        break

                if valid == True:
                    objects.append(rand_bbox)
                    labels.append('_background_')
                    taken = True
    
    for idx_, object_ in enumerate(objects):
        print(labels[idx_])
        try:
            img_object = img[object_[1]:object_[3],object_[0]:object_[2]]
            img_object = cv2.resize(img_object, (224,224))
            counter = counter + 1
            prefix_test= '/home/suda/Desktop/test2/'
            cv2.imwrite( prefix_test+labels[idx_]+'/'+str(counter)+'.jpg',img_object )

        except:
            pass

In [ ]:
#For Training the Model

train_on_gpu = cuda.is_available()
resnet_input =   224 #size of resnet18 input images

classes = ('__background__',
           'aeroplane',
           'bottle','chair'
           )

traindir ='/content/drive/My Drive/dataset1/train2/'
validdir ='/content/drive/My Drive/dataset1/val2/'
testdir = '/content/drive/My Drive/dataset/testset/'

save_file_name = 'resnet18-transfer-4.pt'
checkpoint_path = 'resnet18-transfer-4.pth'

# Change to fit hardware
batch_size = 128

categories = []
img_categories = []
n_train = []
n_valid = []
n_test = []

# Iterate through each category
for d in os.listdir(traindir):
    categories.append(d)

    # Number of each image
    train_imgs = os.listdir(traindir + d)
    valid_imgs = os.listdir(validdir + d)
    test_imgs = os.listdir(testdir + d)
    n_train.append(len(train_imgs))
    n_valid.append(len(valid_imgs))
    n_test.append(len(test_imgs))

    
# Image transformations
image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    # Validation does not use augmentation
    'val':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    # Test does not use augmentation
    'test':
    transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}




data = {
    'train':
    datasets.ImageFolder(root=traindir, transform=image_transforms['train']),
    'val':
    datasets.ImageFolder(root=validdir, transform=image_transforms['val']),
    'test':
    datasets.ImageFolder(root=testdir, transform=image_transforms['test'])
}

# Dataloader iterators
dataloaders = {
    'train': DataLoader(data['train'], batch_size=batch_size, shuffle=True),
    'val': DataLoader(data['val'], batch_size=batch_size, shuffle=True),
    'test': DataLoader(data['test'], batch_size=batch_size, shuffle=True)
}



trainiter = iter(dataloaders['train'])
features, labels = next(trainiter)



learning_rate = 0.001
hyp_momentum = 0.9





In [ ]:
def train(model,
          criterion,
          optimizer,
          train_loader,
          valid_loader,
          save_file_name,
          max_epochs_stop=3,
          n_epochs=20,
          print_every=2):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        save_file_name (str ending in '.pt'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        print_every (int): frequency of epochs to print training stats

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
        history (DataFrame): history of train and validation loss and accuracy
    """

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = timer()

        # Training loop
        for ii, (data, target) in enumerate(train_loader):
            # Tensors to gpu
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()

            # Clear gradients
            optimizer.zero_grad()
            # Predicted outputs are log probabilities
            output = model(data)

            # Loss and backpropagation of gradients
            loss = criterion(output, target)
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()

                    # Forward pass
                    output = model(data)

                    # Validation loss
                    loss = criterion(output, target)
                    # Multiply average loss times the number of examples in batch
                    valid_loss += loss.item() * data.size(0)

                    # Calculate validation accuracy
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    valid_acc += accuracy.item() * data.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                # Calculate average accuracy
                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)

                history.append([train_loss, valid_loss, train_acc, valid_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                    print(
                        f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                    )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
                        )
                        total_time = timer() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict
                        model.load_state_dict(torch.load(save_file_name))
                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_acc',
                                'valid_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    return model, history

In [ ]:

model = models.resnet18(pretrained=True)          #Making of first Model

model.fc = nn.Linear(model.fc.in_features, 4)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
# Update if any errors occur
optimizer = optim.SGD(model.parameters(), learning_rate, hyp_momentum)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)




In [ ]:
class model2(nn.Module):               #Making of second Model
    def __init__(self,model):
        super(model2, self).__init__()
        self.features = nn.Sequential(*list(model.children())[:-3])
        self.fc = nn.Linear(in_features=50176,out_features=4)
        self.softmax = nn.Softmax(dim=1)
        for param in self.features.parameters():
            param.requires_grad = False
    def forward(self,x):
        x=self.features(x)
        x = x.view(x.size(0), -1)
        x=self.fc(x)
        x = self.softmax(x)
        return x
resnet18 = models.resnet18(pretrained=True)
model2= model2(resnet18)
device = "cpu"
model2 = model2.to(device)


#model2.class_to_idx = data['train'].class_to_idx
model2.idx_to_class = {
    idx: class_
    for class_, idx in model.class_to_idx.items()
}
model2.idx_to_class

In [ ]:

if train_on_gpu:
    model = model.to('cuda')
    model2 = model2.to('cuda')



In [ ]:
model, history = train(                       #Train First Model
    model,
    criterion,
    optimizer,
    dataloaders['train'],
    dataloaders['val'],
    save_file_name=save_file_name,
    max_epochs_stop=5,
    n_epochs=30,
    print_every=2)

In [ ]:
model2, history = train(                  #Train Second Model
    model2,
    criterion,
    optimizer,
    dataloaders['train'],
    dataloaders['val'],
    save_file_name=save_file_name,
    max_epochs_stop=5,
    n_epochs=30,
    print_every=2)

In [ ]:
model = torch.load('RES18Final.pt',map_location='cpu')      #First Model
model2 = torch.load('Model3.pt',map_location='cpu')      #Second Model


In [ ]:
def predict_kar(image, model, topk=4):
    
    img = Image.fromarray(image)
    # Resize
    img = img.resize((224, 224))

    # Convert to numpy, transpose color dimension and normalize
    img = np.array(img).transpose((2, 0, 1)) / 256

    # Standardization
    means = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
    stds = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))

    img = img - means
    img = img / stds
    
    # Convert to pytorch tensor
    img_tensor = torch.Tensor(img)
    
    # Resize
    if False:
        img_tensor = img_tensor.view(1, 3, 224, 224).cuda()
    else:
        img_tensor = img_tensor.view(1, 3, 224, 224)

    # Set to evaluation
    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(img_tensor)
        ps = torch.exp(out)
        #ps=out

        # Find the topk predictions
        topk, topclass = ps.topk(topk, dim=1)

        # Extract the actual classes and probabilities
        top_classes = [
            model.idx_to_class[class_] for class_ in topclass.cpu().numpy()[0]
        ]
        top_p = topk.cpu().numpy()[0]

        return img_tensor.cpu().squeeze(), top_p, top_classes

In [ ]:
#window_sizes =[(224,224),(112,112),(112,200),(200,112),(64,64),(96,96),(192,96),(96,192),(64,128),(128,64)]
#window_sizes=[(300,250),(400,250),(200,100),(250,400),(100,200),(150,150),(250,300),(300,300),(100,100)]
window_sizes =[(128,128),(64,64),(64,128),(128,64)]
#window_sizes=[(256,128)]
#window_sizes =[(100,200),(100,100),(200,200),(200,100)]



def pyramid(image, scale=1.5, minSize=(30, 30)):
    yield image
    while True:
        w = int(image.shape[1] / scale)
        image = imutils.resize(image, width=w)
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break   
        yield image

def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[1], stepSize):
        for x in range(0, image.shape[0], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])


In [ ]:

def slid_win(path,step):
    bboxes = []
    image =  cv2.imread(path)
    for (winW,winH) in window_sizes:
        for resized in pyramid(image, scale=2):
            for (x, y, window) in sliding_window(resized, stepSize=step, windowSize=(winW, winH)):
                if window.shape[0] != winH or window.shape[1] != winW:
                    continue


                window=image[x:x+winW,y:y+winH]
                window=cv2.cvtColor(window,cv2.COLOR_BGR2RGB)

                window=cv2.resize(window,(224,224))




                img, top_p, top_classes = predict_kar(window,model,4)
                top_p=top_p/np.sum(top_p)
                score = top_p[0]
                preds = top_classes[0]

                if preds!="__background__":
                    if score > 0.9:
                        bboxes.append({"class":preds,'bbox':[x,y,x+winW,y+winH],'score':score})
    return bboxes

In [ ]:
def slid_win_2d(path,step):
    bboxes = []
    image =  cv2.imread(path)
    for (winW,winH) in window_sizes:
        for resized in pyramid(image, scale=2):
            for (x, y, window) in sliding_window(resized, stepSize=step, windowSize=(winW, winH)):
                if window.shape[0] != winH or window.shape[1] != winW:
                    continue


                window=image[x:x+winW,y:y+winH]
                window=cv2.cvtColor(window,cv2.COLOR_BGR2RGB)

                window=cv2.resize(window,(224,224))




                img, top_p, top_classes = predict_kar(window,model,4)
                top_p=top_p/np.sum(top_p)
                score = top_p[0]
                preds = top_classes[0]
                
                img2, top_p2, top_classes2 = predict_kar(window,model2,4)
                top_p2=top_p2/np.sum(top_p2)
                score2 = top_p2[0]
                preds2 = top_classes2[0]
                                

                if preds!='_background_':
                    if score > 0.9:
                        bboxes.append({"class":preds,'bbox':[x,y,x+winW,y+winH],'score':score})
                        
                if preds2!='_background_':
                    if score > 0.9:
                        bboxes.append({"class":preds2,'bbox':[x,y,x+winW,y+winH],'score':score2})
                        
                        
    return bboxes

In [ ]:

def non_max_suppression_slow(boxes, overlapThresh):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []

	# initialize the list of picked indexes
	pick = []

	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	score = boxes[:,4]
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(score)

	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:
		# grab the last index in the indexes list, add the index
		# value to the list of picked indexes, then initialize
		# the suppression list (i.e. indexes that will be deleted)
		# using the last index
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		suppress = [last]

		# loop over all indexes in the indexes list
		for pos in range(0, last):
			# grab the current index
			j = idxs[pos]

			# find the largest (x, y) coordinates for the start of
			# the bounding box and the smallest (x, y) coordinates
			# for the end of the bounding box
			xx1 = max(x1[i], x1[j])
			yy1 = max(y1[i], y1[j])
			xx2 = min(x2[i], x2[j])
			yy2 = min(y2[i], y2[j])

			# compute the width and height of the bounding box
			w = max(0, xx2 - xx1 + 1)
			h = max(0, yy2 - yy1 + 1)

			# compute the ratio of overlap between the computed
			# bounding box and the bounding box in the area list
			overlap = float(w * h) / area[j]

			# if there is sufficient overlap, suppress the
			# current bounding box
			if overlap > overlapThresh:
				suppress.append(pos)

		# delete all indexes from the index list that are in the
		# suppression list
		idxs = np.delete(idxs, suppress)

	# return only the bounding boxes that were picked
	return boxes[pick]

In [ ]:
def single_layer():
    count = 0                   #Making of boxes[]
    boxes = []
    for i in test_imgs:
      path = voc_dir+i
      l = slid_win(path,40)
      boxes.append({"name": path, " boundary ": l })
      print(count)
      print(boxes[count])
      count = count + 1

        
        
        
    cnt=0                                      #Showing all the boxes
    for i in test_imgs:
        path = voc_dir+i

        img=cv2.imread(path)
        plt.imshow(img)
        plt.show()
        for x in boxes[cnt][" boundary "]:
            plt.gca().add_patch(patches.Rectangle((x['bbox'][0],x['bbox'][1]),x['bbox'][2] - x['bbox'][0],x['bbox'][3] - x['bbox'][1],linewidth=1,edgecolor='r',facecolor='none'))
        plt.imshow(img)
        plt.show()
        cnt=cnt+1

    n_images1= []                   #Making of n_images
    n_images2= []
    n_images3= []

    for i in range(0,len(boxes)):
        length = len(boxes[i][" boundary "])
        if(length == 0):
            continue

        arr1 = np.zeros((length,6))
        arr2 = np.zeros((length,6))
        arr3 = np.zeros((length,6))
        k1=0
        k2=0
        k3=0

        for x in boxes[i][" boundary "]:
            if x['class']=="aeroplane":
                for j in range(0,4):
                    arr1[k1][j] = x['bbox'][j]
                arr1[k1][4]=x['score']
                arr1[k1][5]=1
                k1 = k1+1
            if x['class']=="bottle":
                for j in range(0,4):
                    arr2[k2][j] =x['bbox'][j]
                arr2[k2][4] = x['score']
                arr2[k2][5]=2
                k2 = k2+1
            if x['class']=="chair":
                for j in range(0,4):
                    arr3[k3][j]=x['bbox'][j]
                arr3[k3][4]= x['score']
                arr3[k3][5]=3
                k3 = k3+1  

        if(k1 != 0):
            while(k1<length):
                arr1 = np.delete(arr1, -1 , 0)
                k1 = k1+1
            n_images1.append(( boxes[i]['name'], arr1 )) 

        if k2 != 0 :  
            while(k2<length):
                k2= k2+1
                arr2 = np.delete(arr2, -1  , 0)
            n_images2.append((boxes[i]['name'] , arr2 ))

        if k3 != 0:
            while(k3 < length ):
                arr3 = np.delete(arr3 , -1 , 0)
                k3 = k3 +1
            n_images3.append(( boxes[i]['name'], arr3 ))





        
    d = defaultdict(list)                     #showing boxes after applying NMS
    # construct a list containing the images that will be examined
    # along with their respective bounding boxes
    n_images=[n_images1,n_images2,n_images3]
    cnt=0
    for images in n_images:
        cnt=cnt+1

        # loop over the images
        for (imagePath, boundingBoxes) in images:
           
            # perform non-maximum suppression on the bounding boxes
            pick = non_max_suppression_slow(boundingBoxes, 0.2)
            
            d[imagePath].append(pick)
            cv2.waitKey(0)
    cnt=0
    h=200
    for key, value in d.items():
        h=h+1
        image = cv2.imread(key)
        for pick in value:

            for (startX, startY, endX, endY,score,class_name) in pick:
                cv2.rectangle(image, (int(startX), int(startY)), (int(endX), int(endY)), (0, 0, 255), 2)
        plt.imshow(image)
        vocdir1 ='/home/suda/Desktop/'
        vocdir1+=str(h)+'.jpg'
        plt.savefig(vocdir1)
        plt.show()


In [ ]:
def two_layer():
    count = 0                   #Making of boxes[]
    boxes = []
    for i in test_imgs:
      path = voc_dir+i
      l = slid_win_2d(path,60)
      boxes.append({"name": path, " boundary ": l })
      print(count)
      print(boxes[count])
      count = count + 1

        
        
        
    cnt=0                                      #Showing all the boxes
    for i in test_imgs:
        path = voc_dir+i

        img=cv2.imread(path)
        plt.imshow(img)
        plt.show()
        for x in boxes[cnt][" boundary "]:
            plt.gca().add_patch(patches.Rectangle((x['bbox'][0],x['bbox'][1]),x['bbox'][2] - x['bbox'][0],x['bbox'][3] - x['bbox'][1],linewidth=1,edgecolor='r',facecolor='none'))
        plt.imshow(img)
        plt.show()
        cnt=cnt+1



    n_images1= []                    #Making of n_images
    n_images2= []
    n_images3= []

    for i in range(0,len(boxes)):
        length = len(boxes[i][" boundary "])
        if(length == 0):
            continue

        arr1 = np.zeros((length,6))
        arr2 = np.zeros((length,6))
        arr3 = np.zeros((length,6))
        k1=0
        k2=0
        k3=0

        for x in boxes[i][" boundary "]:
            if x['class']=="aeroplane":
                for j in range(0,4):
                    arr1[k1][j] = x['bbox'][j]
                arr1[k1][4]=x['score']
                arr1[k1][5]=1
                k1 = k1+1
            if x['class']=="bottle":
                for j in range(0,4):
                    arr2[k2][j] =x['bbox'][j]
                arr2[k2][4] = x['score']
                arr2[k2][5]=2
                k2 = k2+1
            if x['class']=="chair":
                for j in range(0,4):
                    arr3[k3][j]=x['bbox'][j]
                arr3[k3][4]= x['score']
                arr3[k3][5]=3
                k3 = k3+1  

        if(k1 != 0):
            while(k1<length):
                arr1 = np.delete(arr1, -1 , 0)
                k1 = k1+1
            n_images1.append(( boxes[i]['name'], arr1 )) 

        if k2 != 0 :  
            while(k2<length):
                k2= k2+1
                arr2 = np.delete(arr2, -1  , 0)
            n_images2.append((boxes[i]['name'] , arr2 ))

        if k3 != 0:
            while(k3 < length ):
                arr3 = np.delete(arr3 , -1 , 0)
                k3 = k3 +1
            n_images3.append(( boxes[i]['name'], arr3 ))




        
    d = defaultdict(list)                     #showing boxes after applying NMS
    # construct a list containing the images that will be examined
    # along with their respective bounding boxes
    n_images=[n_images1,n_images2,n_images3]
    cnt=0
    for images in n_images:
        cnt=cnt+1

        # loop over the images
        for (imagePath, boundingBoxes) in images:
            # load the image and clone it
            image = cv2.imread(imagePath)
            orig = image.copy()

           
            # perform non-maximum suppression on the bounding boxes
            pick = non_max_suppression_slow(boundingBoxes, 0.1)
           
            d[imagePath].append(pick)
            cv2.waitKey(0)
    h=0
    for key, value in d.items():
        h=h+1
        image = cv2.imread(key)
        orig = image.copy()
        for pick in value:
            for (startX, startY, endX, endY,score,class_name) in pick:
                cv2.rectangle(image, (int(startX), int(startY)), (int(endX), int(endY)), (0, 0, 255), 2)
        plt.imshow(image)
        vocdir1 ='/home/suda/Desktop/'
        vocdir1+=str(5)+'.jpg'
        plt.savefig(vocdir1)
        plt.show()

In [ ]:
voc_dir ='/home/suda/Desktop/JPG/'
test_imgs =  os.listdir(voc_dir)
len(test_imgs)

In [ ]:
single_layer()

In [ ]:
two_layer()

In [ ]:
thres_mAP = 0.5
def find_jaccard_overlap(set_1, set_2):
    """
    Find the Jaccard Overlap (IoU) of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: Jaccard Overlap of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # Find intersections
    intersection = find_intersection(set_1, set_2)  # (n1, n2)

    # Find areas of each box in both sets
    areas_set_1 = (set_1[:, 2] - set_1[:, 0]) * (set_1[:, 3] - set_1[:, 1])  # (n1)
    areas_set_2 = (set_2[:, 2] - set_2[:, 0]) * (set_2[:, 3] - set_2[:, 1])  # (n2)

    # Find the union
    # PyTorch auto-broadcasts singleton dimensions
    union = areas_set_1.unsqueeze(1) + areas_set_2.unsqueeze(0) - intersection  # (n1, n2)

    return intersection / union  # (n1, n2)

def find_intersection(set_1, set_2):
    """
    Find the intersection of every box combination between two sets of boxes that are in boundary coordinates.

    :param set_1: set 1, a tensor of dimensions (n1, 4)
    :param set_2: set 2, a tensor of dimensions (n2, 4)
    :return: intersection of each of the boxes in set 1 with respect to each of the boxes in set 2, a tensor of dimensions (n1, n2)
    """

    # PyTorch auto-broadcasts singleton dimensions
    lower_bounds = torch.max(set_1[:, :2].unsqueeze(1), set_2[:, :2].unsqueeze(0))  # (n1, n2, 2)
    upper_bounds = torch.min(set_1[:, 2:].unsqueeze(1), set_2[:, 2:].unsqueeze(0))  # (n1, n2, 2)
    intersection_dims = torch.clamp(upper_bounds - lower_bounds, min=0)  # (n1, n2, 2)
    return intersection_dims[:, :, 0] * intersection_dims[:, :, 1]  # (n1, n2)
def calculate_mAP(det_boxes, det_labels, det_scores, true_boxes, true_labels, true_difficulties):
    """
    Calculate the Mean Average Precision (mAP) of detected objects.

    See https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173 for an explanation

    :param det_boxes: list of tensors, one tensor for each image containing detected objects' bounding boxes
    :param det_labels: list of tensors, one tensor for each image containing detected objects' labels
    :param det_scores: list of tensors, one tensor for each image containing detected objects' labels' scores
    :param true_boxes: list of tensors, one tensor for each image containing actual objects' bounding boxes
    :param true_labels: list of tensors, one tensor for each image containing actual objects' labels
    :param true_difficulties: list of tensors, one tensor for each image containing actual objects' difficulty (0 or 1)
    :return: list of average precisions for all classes, mean average precision (mAP)
    """
    assert len(det_boxes) == len(det_labels) == len(det_scores) == len(true_boxes) == len(
        true_labels) == len(
        true_difficulties)  # these are all lists of tensors of the same length, i.e. number of images
    #n_classes = len(label_map)
    n_classes=4

    # Store all (true) objects in a single continuous tensor while keeping track of the image it is from
    true_images = list()
    for i in range(len(true_labels)):
        true_images.extend([i] * true_labels[i].size(0))
    true_images = torch.LongTensor(true_images)  # (n_objects), n_objects is the total no. of objects across all images
    true_boxes = torch.cat(true_boxes, dim=0)  # (n_objects, 4)
    true_labels = torch.cat(true_labels, dim=0)  # (n_objects)
    true_difficulties = torch.cat(true_difficulties, dim=0)  # (n_objects)
    #print(true_labels.size(0))
    assert true_images.size(0) == true_boxes.size(0) == true_labels.size(0)

    # Store all detections in a single continuous tensor while keeping track of the image it is from
    det_images = list()
    for i in range(len(det_labels)):
        det_images.extend([i] * det_labels[i].size(0))
    det_images = torch.LongTensor(det_images)  # (n_detections)
    det_boxes = torch.cat(det_boxes, dim=0)  # (n_detections, 4)
    det_labels = torch.cat(det_labels, dim=0)  # (n_detections)
    det_scores = torch.cat(det_scores, dim=0)  # (n_detections)

    assert det_images.size(0) == det_boxes.size(0) == det_labels.size(0) == det_scores.size(0)

    # Calculate APs for each class (except background)
    average_precisions = torch.zeros((n_classes - 1), dtype=torch.float)  # (n_classes - 1)
    for c in range(1, n_classes):
        # Extract only objects with this class
        true_class_images = true_images[true_labels == c]  # (n_class_objects)
        true_class_boxes = true_boxes[true_labels == c]  # (n_class_objects, 4)
        true_class_difficulties = true_difficulties[true_labels == c]  # (n_class_objects)
        n_easy_class_objects = (1 - true_class_difficulties).sum().item()  # ignore difficult objects

        # Keep track of which true objects with this class have already been 'detected'
        # So far, none
        true_class_boxes_detected = torch.zeros((true_class_difficulties.size(0)), dtype=torch.uint8)
             # (n_class_objects)

        # Extract only detections with this class
        det_class_images = det_images[det_labels == c]  # (n_class_detections)
        det_class_boxes = det_boxes[det_labels == c]  # (n_class_detections, 4)
        det_class_scores = det_scores[det_labels == c]  # (n_class_detections)
        n_class_detections = det_class_boxes.size(0)
        if n_class_detections == 0:
            continue

        # Sort detections in decreasing order of confidence/scores
        det_class_scores, sort_ind = torch.sort(det_class_scores, dim=0, descending=True)  # (n_class_detections)
        det_class_images = det_class_images[sort_ind]  # (n_class_detections)
        det_class_boxes = det_class_boxes[sort_ind]  # (n_class_detections, 4)

        # In the order of decreasing scores, check if true or false positive
        true_positives = torch.zeros((n_class_detections), dtype=torch.float)  # (n_class_detections)
        false_positives = torch.zeros((n_class_detections), dtype=torch.float)  # (n_class_detections)
        for d in range(n_class_detections):
            this_detection_box = det_class_boxes[d].unsqueeze(0)  # (1, 4)
            this_image = det_class_images[d]  # (), scalar

            # Find objects in the same image with this class, their difficulties, and whether they have been detected before
            object_boxes = true_class_boxes[true_class_images == this_image]  # (n_class_objects_in_img)
            object_difficulties = true_class_difficulties[true_class_images == this_image]  # (n_class_objects_in_img)
            # If no such object in this image, then the detection is a false positive
            if object_boxes.size(0) == 0:
                false_positives[d] = 1
                continue

            # Find maximum overlap of this detection with objects in this image of this class
            overlaps = find_jaccard_overlap(this_detection_box, object_boxes) 
            #print(overlaps)# (1, n_class_objects_in_img)
            max_overlap, ind = torch.max(overlaps.squeeze(0), dim=0)  # (), () - scalars

            # 'ind' is the index of the object in these image-level tensors 'object_boxes', 'object_difficulties'
            # In the original class-level tensors 'true_class_boxes', etc., 'ind' corresponds to object with index...
            original_ind = torch.LongTensor(range(true_class_boxes.size(0)))[true_class_images == this_image][ind]
            # We need 'original_ind' to update 'true_class_boxes_detected'

            # If the maximum overlap is greater than the threshold of 0.5, it's a match
            if max_overlap.item() > thres_mAP:
                # If the object it matched with is 'difficult', ignore it
                if object_difficulties[ind] == 0:
                    # If this object has already not been detected, it's a true positive
                    if true_class_boxes_detected[original_ind] == 0:
                        true_positives[d] = 1
                        true_class_boxes_detected[original_ind] = 1  # this object has now been detected/accounted for
                    # Otherwise, it's a false positive (since this object is already accounted for)
                    else:
                        false_positives[d] = 1
            # Otherwise, the detection occurs in a different location than the actual object, and is a false positive
            else:
                false_positives[d] = 1

        # Compute cumulative precision and recall at each detection in the order of decreasing scores
        cumul_true_positives = torch.cumsum(true_positives, dim=0)  # (n_class_detections)
        cumul_false_positives = torch.cumsum(false_positives, dim=0)  # (n_class_detections)
        cumul_precision = cumul_true_positives / (
                cumul_true_positives + cumul_false_positives + 1e-10)  # (n_class_detections)
        cumul_recall = cumul_true_positives / n_easy_class_objects  # (n_class_detections)

        # Find the mean of the maximum of the precisions corresponding to recalls above the threshold 't'
        recall_thresholds = torch.arange(start=0, end=1.1, step=.1).tolist()  # (11)
        precisions = torch.zeros((len(recall_thresholds)), dtype=torch.float)  # (11)
        for i, t in enumerate(recall_thresholds):
            recalls_above_t = cumul_recall >= t
            if recalls_above_t.any():
                precisions[i] = cumul_precision[recalls_above_t].max()
            else:
                precisions[i] = 0.
        average_precisions[c - 1] = precisions.mean()  # c is in [1, n_classes - 1]

    # Calculate Mean Average Precision (mAP)
    mean_average_precision = average_precisions.mean().item()

    # Keep class-wise average precisions in a dictionary
    average_precisions = {rev_label_map[c + 1]: v for c, v in enumerate(average_precisions.tolist())}

    return average_precisions, mean_average_precision

voc_labels = ('aeroplane', 'bottle','chair')
label_map = {k: v + 1 for v, k in enumerate(voc_labels)}
label_map['background'] = 0
rev_label_map = {v: k for k, v in label_map.items()}

In [ ]:
def single_layer_mAp():

    boxes = []
    
    true_boxes=[]
    true_labels=[]
    true_diffi=[]
        
    det_boxes=[]
    det_labels=[]
    det_scores=[]

    i=0
    counter = 0
    #annot_dir ='/home/suda/Desktop/test/VOC2007/Annotations/'
    image_dir ='/home/suda/Desktop/test/VOC2007/JPEGImages/'
    annot_dir ='/home/suda/Desktop/test/VOC2007/Annot/'
#     annot_dir ='/home/anurag/academics/Sem 6/CS783/hw3/Anot/'
    

    t = glob.iglob(annot_dir+'/*')
    for annot_file in t:
      
        i = i+1
        print("Working on "+str(i))
            
        true_boxes_image=[]
        true_labels_image=[]
        true_diffi_image=[]


      
        file = open(annot_file,'r')
        parsed = xml.parse(file)
        root = parsed.getroot()
      
        for child in root.iter('filename'):
            img_file = child.text
             
        path = image_dir + img_file
#         print(path)
#         img = cv2.imread(image_dir + img_file)
#         img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
      
#         objects = []
#         labels = []
      
        for object in root.iter('object'):
#             print(object[0].text)
            if object[0].text in classes:
                xmin = int(float(object[4][0].text))
                ymin = int(float(object[4][1].text))
                xmax = int(float(object[4][2].text))
                ymax = int(float(object[4][3].text))
                
                
                true_boxes_image.append(torch.Tensor([xmin,ymin,xmax,ymax]))
                true_labels_image.append(model.class_to_idx[object[0].text]) ######### check~~~
#                 print(object[0].text)
#                 print(model.class_to_idx[object[0].text])
                true_diffi_image.append(0)
                
                
        if(len(true_boxes_image)==0):
            print('  No objects in test image#'+np.str(i))
            continue
        else:
            true_boxes_image=torch.stack(true_boxes_image)
            true_diffi_image=torch.Tensor(true_diffi_image)
            true_labels_image=torch.Tensor(true_labels_image) 
            
#             print(true_boxes_image)
#             print(true_diffi_image)
#             print(true_labels_image)

            true_boxes.append(true_boxes_image)
            true_diffi.append(true_diffi_image)
            true_labels.append(true_labels_image)
            
#             print(path)
            l = slid_win(path,50)
            
#             print(l)
#             boxes.append({"name": path, " boundary ": l })
            
            
            bBox={'aeroplane':[],'bottle':[],'chair':[]}
            bBox_mod={'aeroplane':[],'bottle':[],'chair':[]}
            
            for item in l:
                if item['class']=='aeroplane' :
                    bBox['aeroplane'].append([item['bbox'][0],item['bbox'][1],item['bbox'][2],item['bbox'][3], item['score']])
                if item['class']=='bottle' :
                    bBox['bottle'].append([item['bbox'][0],item['bbox'][1],item['bbox'][2],item['bbox'][3], item['score']])          
                if item['class']=='chair' :
                    bBox['chair'].append([item['bbox'][0],item['bbox'][1],item['bbox'][2],item['bbox'][3], item['score']])
    
#             print("printing bBox NOW")
#             print(bBox)
        
        
            for k in bBox:
                bBox[k]=np.asarray(bBox[k])
                bBox_mod[k] =  non_max_suppression_slow(bBox[k],0.3)   ######  NMS ###########
            
#             print()
#             print("printing AFTER NMS")
#             print(bBox_mod)
            det_boxes_image=[]
            det_labels_image=[]
            det_scores_image=[]

            for k in bBox_mod:
                for box in bBox_mod[k]:
                    det_boxes_image.append(torch.Tensor(box[0:4]))
                    det_scores_image.append(box[4])
                    det_labels_image.append(model.class_to_idx[k])

            flag_for_no_detect=0
            for k in bBox_mod:
                if(len(bBox_mod[k])!=0):
                    flag_for_no_detect=1

            if(flag_for_no_detect==0):
                print('  No objects  detected in image#'+np.str(i))
                true_boxes=true_boxes[:-1]
                true_diffi=true_diffi[:-1]
                true_labels=true_labels[:-1]
                continue

            det_boxes_image=torch.stack(det_boxes_image)
            det_scores_image=torch.Tensor(det_scores_image)
            det_labels_image=torch.Tensor(det_labels_image)

            det_boxes.append(det_boxes_image)
            det_scores.append(det_scores_image)
            det_labels.append(det_labels_image)
    # Good formatting when printing the APs for each class and mAP
    
    pp = PrettyPrinter()


    # Calculate mAP
    if(len(det_boxes)==0):
        print("### Nothing detected in all images ####")
    else:
        APs, mAP = calculate_mAP(det_boxes, det_labels, det_scores, true_boxes, true_labels
                             ,true_diffi) 

    # Print AP for each class
    print("done!")
    pp.pprint(APs)

    print('\nMean Average Precision (mAP): %.3f' % mAP)
    
    return

In [ ]:
def double_layer_mAp():

    boxes = []
    
    true_boxes=[]
    true_labels=[]
    true_diffi=[]
        
    det_boxes=[]
    det_labels=[]
    det_scores=[]

    i=0
    counter = 0
    #annot_dir ='/home/suda/Desktop/test/VOC2007/Annotations/'
    image_dir ='/home/suda/Desktop/test/VOC2007/JPEGImages/'
    annot_dir ='/home/suda/Desktop/test/VOC2007/Annot/'
#     annot_dir ='/home/anurag/academics/Sem 6/CS783/hw3/Anot/'

    t = glob.iglob(annot_dir+'/*')
    for annot_file in t:
      
        i = i+1
        print("Working on "+str(i))
            
        true_boxes_image=[]
        true_labels_image=[]
        true_diffi_image=[]


      
        file = open(annot_file,'r')
        parsed = xml.parse(file)
        root = parsed.getroot()
      
        for child in root.iter('filename'):
            img_file = child.text
             
        path = image_dir + img_file
#         print(path)
#         img = cv2.imread(image_dir + img_file)
#         img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
      
#         objects = []
#         labels = []
      
        for object in root.iter('object'):
#             print(object[0].text)
            if object[0].text in classes:
                xmin = int(float(object[4][0].text))
                ymin = int(float(object[4][1].text))
                xmax = int(float(object[4][2].text))
                ymax = int(float(object[4][3].text))
                
                
                true_boxes_image.append(torch.Tensor([xmin,ymin,xmax,ymax]))
                true_labels_image.append(model.class_to_idx[object[0].text]) ######### check~~~
#                 print(object[0].text)
#                 print(model.class_to_idx[object[0].text])
                true_diffi_image.append(0)
                
                
        if(len(true_boxes_image)==0):
            print('  No objects in test image#'+np.str(i))
            continue
        else:
            true_boxes_image=torch.stack(true_boxes_image)
            true_diffi_image=torch.Tensor(true_diffi_image)
            true_labels_image=torch.Tensor(true_labels_image) 
            
#             print(true_boxes_image)
#             print(true_diffi_image)
#             print(true_labels_image)

            true_boxes.append(true_boxes_image)
            true_diffi.append(true_diffi_image)
            true_labels.append(true_labels_image)
            
#             print(path)
            l = slid_win_2d(path,50)
            
#             print(l)
#             boxes.append({"name": path, " boundary ": l })
            
            
            bBox={'aeroplane':[],'bottle':[],'chair':[]}
            bBox_mod={'aeroplane':[],'bottle':[],'chair':[]}
            
            for item in l:
                if item['class']=='aeroplane' :
                    bBox['aeroplane'].append([item['bbox'][0],item['bbox'][1],item['bbox'][2],item['bbox'][3], item['score']])
                if item['class']=='bottle' :
                    bBox['bottle'].append([item['bbox'][0],item['bbox'][1],item['bbox'][2],item['bbox'][3], item['score']])          
                if item['class']=='chair' :
                    bBox['chair'].append([item['bbox'][0],item['bbox'][1],item['bbox'][2],item['bbox'][3], item['score']])
    
#             print("printing bBox NOW")
#             print(bBox)
        
        
            for k in bBox:
                bBox[k]=np.asarray(bBox[k])
                bBox_mod[k] =  non_max_suppression_slow(bBox[k],0.3)   ######  NMS ###########
            
#             print()
#             print("printing AFTER NMS")
#             print(bBox_mod)
            det_boxes_image=[]
            det_labels_image=[]
            det_scores_image=[]

            for k in bBox_mod:
                for box in bBox_mod[k]:
                    det_boxes_image.append(torch.Tensor(box[0:4]))
                    det_scores_image.append(box[4])
                    det_labels_image.append(model.class_to_idx[k])

            flag_for_no_detect=0
            for k in bBox_mod:
                if(len(bBox_mod[k])!=0):
                    flag_for_no_detect=1

            if(flag_for_no_detect==0):
                print('  No objects  detected in image#'+np.str(i))
                true_boxes=true_boxes[:-1]
                true_diffi=true_diffi[:-1]
                true_labels=true_labels[:-1]
                continue

            det_boxes_image=torch.stack(det_boxes_image)
            det_scores_image=torch.Tensor(det_scores_image)
            det_labels_image=torch.Tensor(det_labels_image)

            det_boxes.append(det_boxes_image)
            det_scores.append(det_scores_image)
            det_labels.append(det_labels_image)
    # Good formatting when printing the APs for each class and mAP
    pp = PrettyPrinter()


    # Calculate mAP
    if(len(det_boxes)==0):
        print("### Nothing detected in all images ####")
    else:
        APs, mAP = calculate_mAP(det_boxes, det_labels, det_scores, true_boxes, true_labels
                             ,true_diffi) 

    # Print AP for each class
    print("done!")
    pp.pprint(APs)

    print('\nMean Average Precision (mAP): %.3f' % mAP)
    
    return

In [ ]:
single_layer_mAp()

In [ ]:
double_layer_mAp()

In [ ]:
summary(model2, input_size=(3, 224, 224))

In [ ]:
pp = PrettyPrinter()